<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/BILSTM%2BGLOVE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
# drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
device_name = tf.test.gpu_device_name()
print(device_name)
# df_train = pd.read_csv('/content/SA-emotion-train_val_test_clean_tweets.csv')
df = pd.read_csv('/content/SA-emotion-train_val_test_clean_tweetsekphrasis.csv', sep='\t')
# df_train = df_train.sample(frac=1)
df = df.sample(frac=1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



In [33]:
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [19]:
df.head()

,tweets,emotion
34838,<user> <allcaps> lol </allcaps> ! that is just...,1
10085,still white despite being out for ages in the sun,2
18488,<allcaps> argh </allcaps> ! <repeated> watchin...,0
24045,school <elongated> time . not doing much this ...,1
1393,<user> that ' s right ! i totally forgot . mig...,1


In [22]:
df.emotion.unique()

array([1, 2, 0, 4, 3])

In [26]:
category_dict = {0:[1,0,0,0,0],1:[0,1,0,0,0],2:[0,0,1,0,0],3:[0,0,0,1,0],4:[0,0,0,0,1]}
data_tweet = [x.lower().split() for x in df['tweets']]
data_cat = np.array([category_dict[x] for x in df['emotion']])

data_tweet = data_tweet

In [27]:
test((0.8)*len(data_tweet))

40000

In [30]:
X_train, X_test, y_train, y_test = train_test_split(data_tweet, data_cat, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [32]:
print(len(X_train))
print(len(X_test))
print(len(X_val))

25600
8000
6400


#Glove

In [35]:
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.100d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [36]:
batch_size = 500
Max_input_size = max([len(x) for x in data_tweet])
mul_factor=1.5

In [48]:
X_train = [nltk.pos_tag(x) for x in X_train]

# Multiplication

In [49]:
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='NN':
                tweet[z]=((tweet[z][0]*tweet[i][0])*mul_factor/np.sqrt(tweet[i][0].dot(tweet[i][0])),tweet[z][1])
          if tweet[i][1][:2]=='RB':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]*tweet[i][0])*mul_factor/np.sqrt(tweet[i][0].dot(tweet[i][0])),tweet[z][1])      
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [50]:
epochs=100
Embedding_size=100
model_mul = tf.keras.Sequential()
model_mul.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_mul.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_mul.add(tf.keras.layers.Dropout(0.2))
model_mul.add(tf.keras.layers.Dense(64,activation='relu'))
model_mul.add(tf.keras.layers.Dropout(0.2))
model_mul.add(tf.keras.layers.Dense(5, activation='softmax'))
model_mul.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_mul = model_mul.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history_mul_eval = model_mul.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
51/51 [==============================] - 31s 546ms/step - loss: 1.3809 - accuracy: 0.3328
Epoch 2/40
51/51 [==============================] - 28s 545ms/step - loss: 1.1445 - accuracy: 0.4803
Epoch 3/40
51/51 [==============================] - 28s 548ms/step - loss: 1.0784 - accuracy: 0.5292
Epoch 4/40
51/51 [==============================] - 28s 545ms/step - loss: 1.0517 - accuracy: 0.5425
Epoch 5/40
51/51 [==============================] - 28s 546ms/step - loss: 1.0415 - accuracy: 0.5499
Epoch 6/40
51/51 [==============================] - 28s 547ms/step - loss: 1.0236 - accuracy: 0.5578
Epoch 7/40
51/51 [==============================] - 28s 544ms/step - loss: 1.0121 - accuracy: 0.5656
Epoch 8/40
51/51 [==============================] - 28s 549ms/step - loss: 1.0019 - accuracy: 0.5673
Epoch 9/40
51/51 [==============================] - 28s 550ms/step - loss: 0.9973 - accuracy: 0.5691
Epoch 10/40
51/51 [==============================] - 28s 548ms/step - loss: 0.9895 - accura

IndexError: ignored

#Negative Positive

In [ ]:
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1],i[2]) for i in x if i[0] in vocab.keys() else]
        l=len(tweet)
        for i in range(l):
          tweet[i]=((tweet[i][0]*tweet[i][2]*mul_factor),tweet[i][1],tweet[i][2])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)